### Dataset
This is the paper that contains the data that we will use to develop our model. If you are interested, have a look!

Statistical Analysis and Discovery of Heterogeneous Catalysts Based on Machine Learning from Diverse Published Data


Keisuke Suzuki, Dr. Takashi Toyao, Dr. Zen Maeno, Dr. Satoru Takakusagi, Prof. Ken-ichi Shimizu, Dr. Ichigaku Takigaw
aVolume1 1, Issue 18, 
September 19, 20, 

Pages 

 2019 https://doi.org/10.1002/cctc.201900952

Data paper contains information about the catalyst preparation methods, and the values that needs to be predicted which is the yield of C2 products

### Loading the data
We have already prepared a data for you to download. Run the following code block to download the code

In [9]:
import urllib.request
urllib.request.urlretrieve("https://github.com/GuGroup/NNTutorial-2024-02-11-Winter-camp/raw/main/4NNExampleMaterials/OCM_matrix.csv", "OCM_matrix.csv")
import csv
import torch
import pandas as pd
import torch.nn as nn
df = pd.read_csv('OCM_matrix.csv')
df.head()

,"Y(C2),%",Ag,Al,Ar,Au,B,Ba,Be,Bi,Br,...,"p(O2), bar",p(CH4)/p(O2),"P total, bar","Contact time, s",Impregnation,Mech. mixing,Precipitation,Pyrolysis,Sol-gel,Therm.decomp.
0,5.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.08,4.8,1.0,0.04,1,0,0,0,0,0
1,4.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.08,4.8,1.0,0.04,1,0,0,0,0,0
2,0.4,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.08,4.8,1.0,0.04,1,0,0,0,0,0
3,3.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.08,4.8,1.0,0.04,1,0,0,0,0,0
4,0.1,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,...,0.08,4.8,1.0,0.04,1,0,0,0,0,0


The first column Y(C2),% is the value we want to predict (y), and the rest of the columns are the features.

## Homework part (a)
Instruction:\
Make a Dataset class to load the data.\
HINT: Have a look at HW13

In [112]:
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, random_split

class CatalystData(Dataset):
    def __init__(self, path='OCM_matrix.csv'):
        ####### Fill in here #######
        df = pd.read_csv(path)
        data = df.to_numpy()
        data = torch.tensor(data,dtype=torch.float32)
        self.Y = data[:,:1]
        self.X = data[:,2:]
        
    def __len__(self):
        ####### Fill in here #######
        number_of_data = self.Y.shape[0]
        return number_of_data
        
    def __getitem__(self, idx):
        ####### Fill in here #######
        x = self.X[idx,:]
        y = self.Y[idx,:]
        return x,y
data = CatalystData('OCM_matrix.csv')

## Homework part (b)
Instruction:\
Randomly split your data into train, validation and test by 80, 10, and 10 percent.
HINT: Have a look at HW13

In [113]:
data_train, data_val, data_test = random_split(data,[0.80,0.1,0.1])  ####### Fill in here #######

## Homework part (c)
Instruction:
Calculate the feature and output mean and standard deviation using the training set, and apply the normalization to all data set.


In [114]:
X_train = []
Y_train = []
for x,y in data_train:
    X_train.append(x)
    Y_train.append(y)
X_train = torch.stack(X_train)
Y_train = torch.stack(Y_train)
####### Fill in here #######
Xmean = X_train.mean(0)
Xstd = X_train.std(0)
Ymean = Y_train.mean(0)
Ystd = Y_train.std(0)
####### Fill in here #######
Xstd[Xstd == 0] = 1 # to prevent division by zero
data.X = (data.X-Xmean)/Xstd
data.Y = (data.Y-Ymean)/Ystd

## Homework part (d)
Instruction:\
Construct a simple neural network of your choice

In [115]:
class Model(nn.Module):
    def __init__(self,n_0,n_1,n_2):
        super().__init__()
        ####### Fill in here #######
        self.L1 = nn.Linear(n_0,n_1)
        self.L2 = nn.Linear(n_1,n_2)
        self.act1 = nn.ReLU()
        self.act2 = nn.Sigmoid()
    def forward(self, X):
        ####### Fill in here #######
        Z1 = self.L1(X)
        A1 = self.act1(Z1)
        Z2 = self.L2(A1)
        Z2 = self.act2(Z2)
        return Z2
        
NN = Model(data.X.shape[1],32,1)

## Homework part (e)
Instruction:\
"weight_decay" of the line below is lambda (regularization strength) of the L2 norm.\
optimizer = torch.optim.Adam(NN.parameters(),0.001, weight_decay = 0.001)\
Modify the training code below to perform the optimal lambda parameter search.


In [139]:
dataloader_train = DataLoader(data_train, batch_size=128, shuffle=True)
dataloader_val = DataLoader(data_val, batch_size=128, shuffle=True)
dataloader_test = DataLoader(data_test, batch_size=128, shuffle=True)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(NN.parameters(),0.001, weight_decay = 0.001)
min_val_loss = torch.Tensor([float('Inf')])
train_loss_at_min_val_loss = torch.Tensor([float('Inf')])
for i in range(100): 
    train_loss = 0
    for X, Y in dataloader_train:
        yhat= NN(X)
        optimizer.zero_grad()
        loss = criterion(yhat,Y)
        loss.backward()
        train_loss += loss*Y.shape[0]
        optimizer.step()
    train_loss = train_loss/len(data_train)
    loss_val = 0
    for X, Y in dataloader_val:
        yhat = NN(X)
        loss_val += criterion(yhat,Y)*Y.shape[0]
    loss_val = loss_val/len(data_val)

    if loss_val < min_val_loss:
        torch.save(NN.state_dict(),'best.pth.tar')
        min_val_loss = loss_val
        train_loss_at_min_val_loss = train_loss
print(f'Best model: train_loss {train_loss_at_min_val_loss:.3f} val_loss {min_val_loss:.3f}')


Best model: train_loss 0.642 val_loss 0.808


The code below is used for testing the model with test set.
How is your model performance? 
The RMSE of the b

In [150]:
loss_test = 0
NN.load_state_dict(torch.load('best.pth.tar'))
YTest = []
YhatTest = []
for X, Y in dataloader_test:
    yhat = NN(X)
    loss_test += criterion(yhat,Y)*Y.shape[0]
    YTest.append(Y)
    YhatTest.append(yhat)
loss_test = loss_test/len(data_test)
print(f'Mean squared error: {loss_test:.3f}')
YTest = torch.cat(YTest)
YhatTest = torch.cat(YhatTest)
YTest = YTest*Ystd+Ymean
YhatTest = YhatTest*Ystd+Ymean
print(f'Mean absolute error after denormalizatoin: {torch.mean(torch.abs(YTest-YhatTest)):.3f}')
print(f'Root mean squared error after denormalization: {torch.sqrt(loss_test*Ystd)[0]:.3f}')

Mean squared error: 0.883
Mean absolute error after denormalizatoin: 5.093
Root mean squared error after denormalization: 2.422
